In [ ]:
# Import libraries
import os
import datetime
import pathlib
import requests
import io
import shutil
from glob import glob

# from qgis.core import *

import earthpy as et
import geopandas as gpd
import marxanconpy as mx
import numpy as np
import pandas as pd

import kba_thresh_sa_scripts as ks

# set global cache override variable
CACHE_OVERRIDE = False

In [ ]:
# open full hex.shp file
fullhex_data_path = os.path.join(data_path, "hex_shp", 'Hex_7mi2.shp')
fullhex_layer = gpd.read_file(fullhex_data_path)

# Reproject CRS to ESPG 5070
fullhex_layer_5070 = fullhex_layer.to_crs(epsg='5070')

### Working with qmarxan functions

In [ ]:
# Write formula to create 'bound.dat' file
(# code below taken from 'qmarxan_toolbox_algorithm.py')

# # Constants used to refer to parameters and outputs. They will be
# # used when calling the algorithm from another algorithm, or when
# # calling from the QGIS console.

# PU_LAYER = 'PU_LAYER'
# PU_FIELD = 'PU_FIELD'
# BND_METHOD = 'BND_METHOD'
# BND_TREAT = 'BND_TREAT'
# BND_VALUE = 'BND_VALUE'
# CALC_FIELD = 'CALC_FIELD'
# CALC_METHOD = 'CALC_METHOD'
# TOL = 'TOL'
# OUT_DIR = 'OUT_DIR'

# def create_bound_dat(???self, config???):
#         """
#         Here we define the inputs and output of the algorithm, along
#         with some other properties.
#         """
#         # pu layer
#         self.addParameter(
#                 self.PU_LAYER,
#                 self.tr('Planning unit layer (source for bound.dat file)'),
#                 [QgsProcessing.TypeVectorPolygon]
#             )
#         )
#         # pu id
#         self.addParameter(
#             QgsProcessingParameterField(
#                 self.PU_FIELD,
#                 self.tr('Planning unit id field'),
#                 parentLayerParameterName=self.PU_LAYER,
#                 type=QgsProcessingParameterField.Numeric
#             )
#         )
#         #
#         # advanced settings
#         #
#         #  bnd method
#         bndMethod = QgsProcessingParameterEnum(
#             self.BND_METHOD,
#             self.tr('Boundary method (how lengths between planning units will be set)'),
#             options = ["Single","Measured","Weighted","Field"],
#             defaultValue = 0
#         )
#         bndMethod.setFlags(bndMethod.flags() | QgsProcessingParameterDefinition.FlagAdvanced)
#         self.addParameter(bndMethod)
#         # bnd treatment
#         bndTreatment = QgsProcessingParameterEnum(
#             self.BND_TREAT,
#             self.tr('Boundary treatment (how values for PUs on perimeter of study area will be set)'),
#             options = ["Full Value","Half Value","Exclude"],
#             defaultValue = 0
#         )
#         bndTreatment.setFlags(bndTreatment.flags() | QgsProcessingParameterDefinition.FlagAdvanced)
#         self.addParameter(bndTreatment)
#         # single value
#         bndValue = QgsProcessingParameterNumber(
#             self.BND_VALUE,
#             self.tr('Boundary value (value for all boundaries regardless of measured length)'),
#             type=QgsProcessingParameterNumber.Integer, 
#             minValue=0, 
#             defaultValue=1,
#             optional=True
#         )
#         bndValue.setFlags(bndValue.flags() | QgsProcessingParameterDefinition.FlagAdvanced )
#         self.addParameter(bndValue)
#         # calculation field
#         calcField = QgsProcessingParameterField(
#             self.CALC_FIELD,
#             self.tr('Calculation field (field to weight or assign boundary lengths)'),
#             parentLayerParameterName=self.PU_LAYER,
#             type=QgsProcessingParameterField.Numeric,
#             optional = True
#         )
#         calcField.setFlags(calcField.flags() | QgsProcessingParameterDefinition.FlagAdvanced)
#         self.addParameter(calcField)
#         # calculation method
#         calcMethod = QgsProcessingParameterEnum(
#             self.CALC_METHOD,
#             self.tr('Calculation method (how to assign boundary length if values between adjacent planning units differ)'),
#             options = ["Mean","Maximum","Minimum"],
#             defaultValue = 0,
#             optional = True
#         )
#         calcMethod.setFlags(calcMethod.flags() | QgsProcessingParameterDefinition.FlagAdvanced)
#         self.addParameter(calcMethod)
#         # rounding precision
#         tolerance = QgsProcessingParameterEnum(
#             self.TOL,
#             self.tr('Export precision tolerance (in map units)'),
#             options = ["100","10","1","0.1","0.01","0.001","0.0001","0.00001"],
#             defaultValue = 3
#         )
#         tolerance.setFlags(tolerance.flags() | QgsProcessingParameterDefinition.FlagAdvanced)
#         self.addParameter(tolerance)

#         # select output folder
#         defDir = os.path.join(os.path.expanduser('~'),'marxanproj1','input')
#         self.addParameter(
#             QgsProcessingParameterFolderDestination(
#                 self.OUT_DIR,
#                 self.tr('Marxan input folder (place to write bound.dat file)'),
#                 defDir,
#                 optional=False
#             )
#         )
#   

In [ ]:
# for bound.dat
# Code pasted from 'qmarxan_toolbox_algorithm' (lines ~500-765) 
# https://github.com/tsw-apropos/qmarxantoolbox/blob/master/QMarxanToolbox/qmarxan_toolbox_algorithm.py

# create temporary file names 
        tempsegfile = 'tempsegfile_%s.txt' % os.getpid()
        tempsortedfile = 'tempsortedfile_%s.txt' % os.getpid()
        tempadjfile = 'tempadjfile_%s.txt' % os.getpid()
        tempsortedadjfile = 'tempsortedadjfile_%s.txt' % os.getpid()
        errorlog = 'topo_error_log_%s.txt' % datetime.date.today().isoformat()
        # get field indexes for puid and boundary fields
        puIdx = self.puLayer.dataProvider().fields().indexFromName(self.puField)
        if self.bndField != None:
            fldIdx = self.puLayer.dataProvider().fields().indexFromName(self.bndField)
        else:
            fldIdx = -1


# step 1 - build temporary segment file and dictionary
#         #
#         # notify users
#         feedback.setProgress(0)
        # set values
        tsf = open(tempsegfile,'w')
        inGeom = QgsGeometry()
        segLineCnt = 0
        # loop through features
        lineCount = 0
        fCount = self.puLayer.dataProvider().featureCount()
#         x = 0
#         progPct = 0
#         progMin = 0
#         progMax = 30
#         progPct = progMin
#         lastPct = progPct
#         progRange = progMax - progMin
        for feat in self.puLayer.getFeatures():
#             x += 1
#             progPct = ((float(x) / float(fCount) * 100) * (progRange/100.0)) + progMin
#             if int(progPct) > lastPct:
#                 feedback.setProgress(progPct)
#                 lastPct = progPct
            attr = feat.attributes()
            pid = int(attr[puIdx])
            if fldIdx != -1:
                cost = str(attr[fldIdx])
            else:
                cost = '1.0'
            inGeom = feat.geometry()
            pointList = extractPoints(inGeom)
            prevPoint = 0
            for i in pointList:
                if prevPoint == 0:
                    prevPoint = i
                else:
                    # write line segment
                    segLen = LineLength([prevPoint[0],prevPoint[1]], [i[0],i[1]])
                    # make spatial key to segment file
                    if round(float(prevPoint[0]),self.tol) < round(float(i[0]),self.tol) or \
                        (round(float(prevPoint[0]),self.tol) == round(float(i[0]),self.tol) \
                        and round(float(prevPoint[1]),self.tol) < round(float(i[1]),self.tol) ):
                        skey = str(round(float(prevPoint[0]),self.tol)) + '|' + \
                            str(round(float(prevPoint[1]),self.tol)) + '|' + \
                            str(round(float(i[0]),self.tol)) + '|' +  \
                            str(round(float(i[1]),self.tol))
                    else:
                        skey = str(round(float(i[0]),self.tol)) + '|' +  \
                            str(round(float(i[1]),self.tol)) + '|' + \
                            str(round(float(prevPoint[0]),self.tol)) + '|' + \
                            str(round(float(prevPoint[1]),self.tol))
                    if segLen > 0:
                        outLine = '%s,%d,%f,%f\n' %  (skey, int(pid), float(cost), segLen)
                        tsf.write(outLine)
                        lineCount += 1
                    prevPoint = i
        # clean up
        tsf.close()
        # sort the file
        self.batch_sort(tempsegfile, tempsortedfile)
        os.remove(tempsegfile)
        #
        # step 2 - loop through sorted file and create adjacency file
        #    
        # notify users
        # 
        tsf = open(tempsortedfile,'r')
        taf = open(tempadjfile,'w')
        done = False
        pl = ''
        x = 0
        adjFileLen = 0
#         progMin = 35
#         progMax = 65
#         progPct = progMin
#         lastPct = progPct
#         progRange = progMax - progMin
        while not done:
#             x += 1
#             progPct = ((float(x) / float(lineCount) * 100) * (progRange/100.0)) + progMin
#             if int(progPct) > lastPct:
#                 feedback.setProgress(progPct)
#                 lastPct = progPct
            line = tsf.readline()
            if line == '':
                done = True
            else:
                cl = line.rstrip().split(',')
            if pl != '' and pl != ['']:
                if cl != '' and pl[0] == cl[0]:
                    fCost = 1
                    if self.bndMethod == 'Single':
                        fCost = str(self.bndValue)
                    elif self.bndMethod == 'Field':
                        bCost = 1
                        if float(pl[2])== float(cl[2]):
                            bCost = float(pl[2])
                        else:
                            if self.calcMethod == 'Maximum':
                                bCost = max([float(pl[2]),float(cl[2])])
                            elif self.calcMethod == 'Minimum':
                                bCost = min([float(pl[2]),float(cl[2])])
                            else:
                                bCost = (float(pl[2]) + float(cl[2]))/2.0
                        fCost = str(bCost)
                    elif self.bndMethod  == 'Weighted':
                        bCost = 1
                        if float(pl[2])== float(cl[2]):
                            bCost = float(pl[2])
                        else:
                            if self.calcMethod == 'Maximum':
                                bCost = max([float(pl[2]),float(cl[2])])
                            elif self.calcMethod == 'Minimum':
                                bCost = min([float(pl[2]),float(cl[2])])
                            else:
                                bCost = sum([float(pl[2]),float(cl[2])])/2.0
                        fCost = str(float(pl[3]) * bCost)
                    else:
                        fCost = str(pl[3])
                    # topology error test
                    # check for more matching lines
                    errorLines = True
                    topologyErrorFound = False
                    pids = ''
                    while errorLines:
                        line = tsf.readline()
                        chkLine = line.rstrip().split(',')
                        if chkLine != '' and chkLine[0] == pl[0]:
                            topologyErrorFound = True
                            # an error exists
                            if pids == '':
                                pids = str(pl[1]) + ',' + str(cl[1]) + ',' + str(chkLine[1])
                            else:
                                pids = pids + ',' + str(chkLine[1])
                        else:
                            errorLines = False
                    if topologyErrorFound:
                        if topoErrorCount == 0:
                            el = open(errorlog, 'w')
                            outline = 'There should never be more than 2 overlapping ' + \
                                'line segments. \n' + \
                                'Below are listed cases where more than 2 have ' + \
                                'been identified. \n' + 'These should all be ' + \
                                'corrected before using the boundary file\n' + \
                                '-------\n' 
                            el.write(outline)
                        outline = 'Line segments defined as %s may be topologically invalid.\n' % (str(pl[0]))
                        outline = outline + 'Area ids %s appear to overlap.\n--\n' % (pids) 
                        el.write(outline)
                        topoErrorCount += 1
                    else:
                        # no error proceed
                        if int(pl[1]) < int(cl[1]):
                            taf.write('%020d,%020d,%s\n' % (int(pl[1]),int(cl[1]),fCost))
                        else:
                            taf.write('%020d,%020d,%s\n' % (int(cl[1]),int(pl[1]),fCost))
                        adjFileLen += 1
                elif type(pl) == list:
                    fCost = 1
                    if self.bndMethod == 'Single':
                        fCost = str(self.bndValue)
                    elif self.bndMethod  == 'Field':
                        fCost = str(pl[2])
                    elif self.bndMethod  == 'Weighted':
                        fCost = str(float(pl[3]) * float(pl[2]))
                    else:
                        fCost = str(pl[3])
                    taf.write('%020d,%020d,%s\n' % (int(pl[1]),int(pl[1]),fCost))
            pl = line.rstrip().split(',')
        tsf.close()
        taf.close()
        os.remove(tempsortedfile)
        # sort adjacency file
        self.batch_sort(tempadjfile, tempsortedadjfile)
        os.remove(tempadjfile)
        #
        # step 3 - write boundary file
        #
        # notify users
        #
        saf = open(tempsortedadjfile,'r')
        faf = open(self.outFName,'w')
        faf.write("id1\tid2\tboundary\n")
        done = False
        pl = ''
#         x = 0
#         progMin = 70
#         progMax = 99
#         progPct = progMin
#         lastPct = progPct
#         progRange = progMax - progMin
        while not done:
#             x += 1
#             progPct = ((float(x) / float(adjFileLen) * 100) * (progRange/100.0)) + progMin
#             if int(progPct) > lastPct:
#                 feedback.setProgress(progPct)
#                 lastPct = progPct
            line = saf.readline()
            if line == '':
                done = True
                cl = ''
            else:
                cl = line.rstrip().split(',')
            if pl != '':
                if cl != '' and pl[0] == cl[0] and pl[1] == cl[1]:
                    if self.bndMethod  == 'Measured' or self.bndMethod == 'Weighted':
                        # NOTE: 
                        # If weighted or measured methods are used
                        # then all the segments' lengths are added together
                        #
                        # If Single or Field methods are used
                        # then only the first rows values are used as all 
                        # other rows are redundant
                        pl = [pl[0],pl[1],sum([float(pl[2]),float(cl[2])])]
                else:
                    bound = adjBound(float(pl[2]),pl[0],pl[1])
                    if self.bndMethod  in ('Field','Weighted'):
                        boundStr = str(bound)
                    else:
                        boundStr = str(round(float(bound),self.tol))
                    if float(bound) > 0.0:
                        faf.write('%d\t%d\t%s\n' % (int(pl[0]),int(pl[1]),boundStr))
                    pl = line.rstrip().split(',')
            else:
                pl = cl
        saf.close()
        faf.close()
        os.remove(tempsortedadjfile)
        if topoErrorCount > 0:
            el.close()
            messageText = '%d possible topological error(s) found. ' % topoErrorCount
            messageText += 'Please check error log in same directory as boundary file.'
            feedback.pushInfo(messageText)
            return {'Status':'Error'}
        else:
            feedback.pushInfo('Boundary file successfully written')
            return {'Status':'Success'}


FUNCTION TO CREATE PUVSP.DAT & SPEC.DAT (PUVSP_SPORDER.DAT not needed, as currently we are looking at only one ecosystem at a time)

find code to replicate the Zonal Histogram tool in QGIS (Processing Toolbox > Raster Analysis > Zonal Histogram) https://docs.qgis.org/3.22/en/docs/user_manual/processing_algs/qgis/rasteranalysis.html#zonal-histogram import processing processing.run("algorithm_id", {parameter_dictionary})
I think this comes from pyQGIS, but I don't think that is included with the earth-analytics-python-env??

Here's info on running pyQGIS in Jupyter https://lerryws.xyz/posts/PyQGIS-in-Jupyter-Notebook

here's my initial guess, using parameters copied from QGIS 'Zonal Histograms' log - from qgis.core import processing processing.run("native:zonalhistogram", { 'COLUMN_PREFIX' : '', 'INPUT_RASTER' : 'F:/NatureServe/LanasData/raster/foothill_r.tif', 'INPUT_VECTOR' : 'F:/NatureServe/LanasData/marxan_prep/foothill/pulayercws.shp', 'OUTPUT' : 'F:/NatureServe/pulayerfeatures.shp', 'RASTER_BAND' : 1 })

add column, multiplying pixel count by raster pixel area variable (our data's is 900, 30m x 30m = 900 sq m/pixel), this gives total extent of ecosystem within each individual planning unit hex of the hexfile.shp (ex. if pixelcount = 5, area = 4500, or 5 x 900)

use this as the source info for qmarxan 'export feature files' function input parameters copied from QGIS - Input parameters: { 'FEAT_FIELDS' : ['7147'], 'OUT_DIR' : 'F:\NatureServe\524 QM test\input', 'PU_FIELD' : 'PUID', 'PU_LAYER' : 'F:/NatureServe/pulayerfeatures.shp' }

In [ ]:
# WRITE FUNCTION TO CREATE PUVSP.DAT & SPEC.DAT
# (PUVSP_SPORDER.DAT not needed, as currently we are working with just one one 
# ecosystem at a time, rather than looking at mulitiple conservation features 
# in a single run)

# find code to replicate the Zonal Histogram tool in QGIS 
# (Processing Toolbox > Raster Analysis > Zonal Histogram) 
# https://docs.qgis.org/3.22/en/docs/user_manual/processing_algs/qgis/
# rasteranalysis.html#zonal-histogram 
# import processing processing.run("algorithm_id", {parameter_dictionary})

# I think the 'import processing' code suggestion comes from pyQGIS, but I 
# don't think that is included with the earth-analytics-python-env??

# Here's info on running pyQGIS in Jupyter 
# https://lerryws.xyz/posts/PyQGIS-in-Jupyter-Notebook

# here's my initial guess at some of the pyQGIS code, using parameters copied 
# from the QGIS 'Zonal Histograms' log window - 
from qgis.core import processing processing.run(
    "native:zonalhistogram", { 
        'COLUMN_PREFIX' : '', 
        'INPUT_RASTER' : 'F:/NatureServe/LanasData/raster/foothill_r.tif', 
        'INPUT_VECTOR' : 'F:/NatureServe/LanasData/marxan_prep/foothill/pulayercws.shp', 
        'OUTPUT' : 'F:/NatureServe/pulayerfeatures.shp', 
        'RASTER_BAND' : 1 
    })

# the result will show the number of raster pixels within each hexgrid cell.

# add new column, multiplying this pixel count by the raster pixel area 
# variable, to give the total extent of ecosystem within each individual 
# planning unit hex cell.
# the pixel area can be determined by looking at the raster saved to the 
# 'source_data' directory in an earlier formula. Maybe a new function should 
# be written to get this value, and that function would be called within this
# current 'create_puvsp_dat' function? 
# (our data's pixel area is 900, as 30m x 30m = 900 sq m/pixel. If the 
# pixelcount = 5, area = 4500, or 5 x 900)

# use this table (dataframe?) as the source info for qmarxan 'export feature
# files' function (WHICH WOULD ALSO CREATE THE SPEC.DAT FILE)

# input parameters copied from QGIS 'export_features_files' log window - 
Input parameters: { 
    'FEAT_FIELDS' : ['7147'], 
    'OUT_DIR' : 'F:\NatureServe\524 QM test\input', 
    'PU_FIELD' : 'PUID', 
    'PU_LAYER' : 'F:/NatureServe/pulayerfeatures.shp' 
}

# or try this -
# Zonal Statistics Algorithm with Python in 4 Steps
# How to summarize raster data for polygon zones
# https://towardsdatascience.com/zonal-statistics-algorithm-with-python-in-4-steps-382a3b66648a

In [ ]:
from .qmarxan_utils import runMarxanOnce